# Breast Cancer Detection Model Ver01

In [5]:
import os
import cv2
import openslide
import numpy
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K

In [ ]:
pos_slide_path = '../data/train/16-S-042893_A1.mrxs'
neg_slide_path = '../data/train/'